<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz.

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno: Felipe Jorquera Díaz


### **Link de repositorio de GitHub:** `https://github.com/felipejorqueradiaz/MDS7202-Laboratorios`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 23/06/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasificación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [3]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
df_comics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1285 entries, 0 to 1366
Data columns (total 82 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        1285 non-null   int64  
 1   name                              1285 non-null   object 
 2   real_name                         1164 non-null   object 
 3   full_name                         895 non-null    object 
 4   overall_score                     1285 non-null   object 
 5   history_text                      1285 non-null   object 
 6   powers_text                       961 non-null    object 
 7   intelligence_score                1285 non-null   int64  
 8   strength_score                    1285 non-null   int64  
 9   speed_score                       1285 non-null   int64  
 10  durability_score                  1285 non-null   int64  
 11  power_score                       1285 non-null   int64  
 12  combat

In [6]:
df_comics.isna().sum().sort_values(ascending=False).head(50)

skin_color                          1131
place_of_birth                       551
base                                 472
relatives                            456
full_name                            390
occupation                           347
type_race                            343
powers_text                          324
eye_color                            194
hair_color                           182
first_appearance                     143
real_name                            121
creator                              119
gender                                94
img                                   70
has_immortality                       57
has_regeneration                      57
has_telepathy                         57
has_toxin_and_disease_resistance      57
has_telekinesis                       57
has_force_fields                      57
has_magic                             57
has_cold_resistance                   57
has_energy_manipulation               57
has_energy_absor

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [7]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [8]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [9]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [11]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [12]:
#Seteamos el idioma en inglés
stop_words = stopwords.words('english')

class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

In [13]:
transformadores = ColumnTransformer(
            transformers = [
                ('MinMax', MinMaxScaler(),  ['intelligence_score', 'strength_score', 'speed_score',
                                             'durability_score', 'power_score', 'combat_score']),
                ('BagOfWords', CountVectorizer(tokenizer= StemmerTokenizer(),
                                               ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                                              ), 'history_text')
            ])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [15]:
pipeline_MultiNB = Pipeline([
    ('preprocesamiento', transformadores),
    ('percentiles', SelectPercentile(f_classif, percentile = 90)),
    ('modelo', MultinomialNB())
     ])

In [16]:
pipeline_DummyClas = Pipeline([
    ('preprocesamiento', transformadores),
    ('percentiles', SelectPercentile(f_classif, percentile = 90)),
    ('modelo', DummyClassifier(strategy="most_frequent"))
     ])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
                                df_comics.drop('alignment', axis=1),
                                df_comics['alignment'],
                                random_state = 0,
                                stratify = df_comics['alignment'],
                                test_size = 0.2)

In [18]:
pipeline_MultiNB.fit(X_train, y_train)
y_pred = pipeline_MultiNB.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.71      0.14      0.23        86
        Good       0.60      0.97      0.74       148
     Neutral       1.00      0.04      0.08        23

    accuracy                           0.61       257
   macro avg       0.77      0.39      0.35       257
weighted avg       0.67      0.61      0.51       257



In [19]:
pipeline_DummyClas.fit(X_train, y_train)
y_pred = pipeline_DummyClas.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.00      0.00      0.00        86
        Good       0.58      1.00      0.73       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.58       257
   macro avg       0.19      0.33      0.24       257
weighted avg       0.33      0.58      0.42       257



C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Respuesta:

El clasificador Naive Bayes tiene la simpleza de que no recibe ningún tipo de parámetro de entrada, por lo que no necesita un optimizador de hiperparámetros ni ningún tipo de cambio especial. Esto lo hace uno de los modelos más simples de aplicar. Sin embargo, es esta simpleza la que también le puede jugar en contra ya que es difícil que pueda adaptarse lo suficientemente bien a ciertos datos. 

En este caso comparamos un Naive Bayes Multinomial con una decisión aleatoria y el resultado es bastante similar en términos de accuracy (también en recall en este caso, pero puede ir variando según el random_state), lo que implica que MultiNB no es tan eficiente como modelo a la hora de clasificar y se necesita un clasificador más potente y que posea hiperparámetros para así obtener el mejor perfomance.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [20]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV

In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
pipeline_Grid = Pipeline([
    ('preprocesamiento', transformadores),
    ('percentiles', SelectPercentile(f_classif, percentile = 90)),
    ('modelo', MultinomialNB())
     ])

In [27]:
params = [
    {
        'modelo': [LogisticRegression(random_state = 0)],
        'modelo__penalty': ['l1', 'l2'],
        'modelo__solver': ['liblinear'],
        'preprocesamiento__BagOfWords__ngram_range': [(1,1), (1,2), (1,3)],
        'percentiles__percentile': [20, 40, 60, 80]
    },{
        'modelo': [RandomForestClassifier(random_state=0)],
        'modelo__max_depth': [2,4,6],
        'preprocesamiento__BagOfWords__ngram_range': [(1,1), (1,2), (1,3)],
        'percentiles__percentile': [20, 40, 60, 80]
    },{
        'modelo': [KNeighborsClassifier()],
        'modelo__n_neighbors': [2,3,4,5,6],
        'preprocesamiento__BagOfWords__ngram_range': [(1,1), (1,2), (1,3)],
        'percentiles__percentile': [20, 40, 60, 80]
    }]

##### CUIDADO CON LA EJECUCIÓN DE ESTA CELDA POR FAVOR, DEJÉ EL MODELO GUARDADO EN UN PICKLE PARA QUE NO TENER QUE RE-ENTRENAR

In [28]:
grid_model = HalvingGridSearchCV(pipeline_Grid,
                           param_grid=params,
                           verbose=10)
grid_model.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 30
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 120
n_resources: 30
Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5; 1/120] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 1)
[CV 1/5; 1/120] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 1);, score=(train=1.000, test=0.667) total time=   0.4s
[CV 2/5; 1/120] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 1)
[CV 2/5; 1/120] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__p

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


[CV 5/5; 1/120] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 1);, score=(train=1.000, test=0.800) total time=   0.2s
[CV 1/5; 2/120] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 2)
[CV 1/5; 2/120] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 2);, score=(train=1.000, test=0.667) total time=   0.5s
[CV 2/5; 2/120] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 2)
[CV 2/5; 2/120] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preproces

C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


[CV 5/5; 2/120] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 2);, score=(train=1.000, test=0.800) total time=   0.3s
[CV 1/5; 3/120] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 3)
[CV 1/5; 3/120] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 3);, score=(train=1.000, test=0.667) total time=   0.6s
[CV 2/5; 3/120] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 3)
[CV 2/5; 3/120] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, percentiles__percentile=20, preproces

...

[CV 4/5; 4/14] END modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 2);, score=(train=0.606, test=0.585) total time=   5.7s
[CV 5/5; 4/14] START modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 2)
[CV 5/5; 4/14] END modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 2);, score=(train=0.634, test=0.585) total time=   4.5s
[CV 1/5; 5/14] START modelo=RandomForestClassifier(random_state=0), modelo__max_depth=6, percentiles__percentile=80, preprocesamiento__BagOfWords__ngram_range=(1, 2)
[CV 1/5; 5/14] END modelo=RandomForestClassifier(random_state=0), modelo__max_depth=6, percentiles__percentile=80, preprocesamiento__BagOfWords__ngram_range=(1, 2);, score=(train=0.628, test=0.556) total time=   5.0s
[CV 2/5; 5/14] START modelo=RandomForestClassifier(random_s

[CV 2/5; 9/14] END modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 3);, score=(train=0.586, test=0.556) total time=   5.6s
[CV 3/5; 9/14] START modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 3)
[CV 3/5; 9/14] END modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 3);, score=(train=0.577, test=0.500) total time=   5.6s
[CV 4/5; 9/14] START modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 3)
[CV 4/5; 9/14] END modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles__percentile=20, preprocesamiento__BagOfWords__ngram_range=(1, 3);, score=(train=0.583, test=0.585) total time=   5.6s
[CV 5/5; 9/14] START modelo=KNeighborsClassifier(), modelo__n_neighbors=3, percentiles_

[CV 5/5; 13/14] END modelo=KNeighborsClassifier(), modelo__n_neighbors=5, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 3);, score=(train=0.588, test=0.585) total time=   5.7s
[CV 1/5; 14/14] START modelo=KNeighborsClassifier(), modelo__n_neighbors=6, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 2)
[CV 1/5; 14/14] END modelo=KNeighborsClassifier(), modelo__n_neighbors=6, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 2);, score=(train=0.609, test=0.556) total time=   4.1s
[CV 2/5; 14/14] START modelo=KNeighborsClassifier(), modelo__n_neighbors=6, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 2)
[CV 2/5; 14/14] END modelo=KNeighborsClassifier(), modelo__n_neighbors=6, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 2);, score=(train=0.577, test=0.556) total time=   5.8s
[CV 3/5; 14/14] START modelo=KNeighborsClassifier(), modelo__n_neighbors=6, percen

[CV 3/5; 4/5] END modelo=KNeighborsClassifier(), modelo__n_neighbors=5, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 3);, score=(train=0.586, test=0.580) total time=  19.2s
[CV 4/5; 4/5] START modelo=KNeighborsClassifier(), modelo__n_neighbors=5, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 3)
[CV 4/5; 4/5] END modelo=KNeighborsClassifier(), modelo__n_neighbors=5, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 3);, score=(train=0.585, test=0.596) total time=  19.3s
[CV 5/5; 4/5] START modelo=KNeighborsClassifier(), modelo__n_neighbors=5, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 3)
[CV 5/5; 4/5] END modelo=KNeighborsClassifier(), modelo__n_neighbors=5, percentiles__percentile=40, preprocesamiento__BagOfWords__ngram_range=(1, 3);, score=(train=0.565, test=0.578) total time=  29.1s
[CV 1/5; 5/5] START modelo=KNeighborsClassifier(), modelo__n_neighbors=6, percentiles__perce

HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocesamiento',
                                               ColumnTransformer(transformers=[('MinMax',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_score',
                                                                                 'combat_score']),
                                                                               ('BagOfWords',
                                                           

In [29]:
import pickle

In [50]:
# Esta celda guarda el modelo en el pickle, por lo que se ha comentado.
'''
with open('grid_model.pickle', 'wb') as handle:
    pickle.dump(grid_model, handle, protocol=pickle.HIGHEST_PROTOCOL)
'''

"\nwith open('grid_model.pickle', 'wb') as handle:\n    pickle.dump(grid_model, handle, protocol=pickle.HIGHEST_PROTOCOL)\n"

In [31]:
with open('grid_model.pickle', 'rb') as handle:
    grid_model = pickle.load(handle)

In [32]:
y_pred = grid_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.00      0.00      0.00        86
        Good       0.58      1.00      0.73       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.58       257
   macro avg       0.19      0.33      0.24       257
weighted avg       0.33      0.58      0.42       257



C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Felipe\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
pd.DataFrame(grid_model.cv_results_).sort_values('rank_test_score', ascending = True).head(20)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_modelo,param_modelo__penalty,param_modelo__solver,param_percentiles__percentile,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
115,0,30,0.342891,0.120256,0.090346,0.065709,KNeighborsClassifier(n_neighbors=6),NaN,NaN,60,...,0.673333,0.152607,1,0.652174,0.478261,0.608696,0.666667,0.625000,0.606159,0.067079
84,0,30,0.319721,0.116841,0.091145,0.069558,KNeighborsClassifier(n_neighbors=6),NaN,NaN,20,...,0.673333,0.213333,1,0.826087,0.521739,0.608696,0.666667,0.708333,0.666304,0.101504
119,0,30,0.381760,0.123010,0.085810,0.068413,KNeighborsClassifier(n_neighbors=6),NaN,NaN,80,...,0.673333,0.152607,1,0.652174,0.478261,0.608696,0.666667,0.625000,0.606159,0.067079
118,0,30,0.357498,0.113097,0.089441,0.069084,KNeighborsClassifier(n_neighbors=6),NaN,NaN,80,...,0.673333,0.152607,1,0.608696,0.478261,0.608696,0.666667,0.666667,0.605797,0.068837
116,0,30,0.387772,0.133169,0.094329,0.068302,KNeighborsClassifier(n_neighbors=6),NaN,NaN,60,...,0.673333,0.152607,1,0.652174,0.478261,0.608696,0.666667,0.625000,0.606159,0.067079
96,0,30,0.324265,0.115351,0.083775,0.068607,KNeighborsClassifier(n_neighbors=6),NaN,NaN,20,...,0.640000,0.130639,6,0.695652,0.478261,0.608696,0.708333,0.666667,0.631522,0.083974
49,0,30,0.471111,0.129804,0.099977,0.067461,RandomForestClassifier(random_state=0),NaN,NaN,20,...,0.640000,0.130639,6,1.000000,1.000000,0.956522,0.958333,1.000000,0.982971,0.020864
97,0,30,0.358799,0.120367,0.090420,0.066274,KNeighborsClassifier(n_neighbors=6),NaN,NaN,20,...,0.640000,0.130639,6,0.608696,0.521739,0.608696,0.666667,0.625000,0.606159,0.047235
98,0,30,0.465248,0.175722,0.114155,0.075358,KNeighborsClassifier(n_neighbors=6),NaN,NaN,20,...,0.640000,0.130639,6,0.565217,0.478261,0.608696,0.666667,0.625000,0.588768,0.064101
28,0,30,0.499567,0.124468,0.099897,0.072386,RandomForestClassifier(random_state=0),NaN,NaN,40,...,0.640000,0.130639,6,0.739130,0.956522,0.695652,0.833333,0.833333,0.811594,0.090158


In [49]:
pd.DataFrame(grid_model.cv_results_).iloc[115]

iter                                                                                               0
n_resources                                                                                       30
mean_fit_time                                                                               0.342891
std_fit_time                                                                                0.120256
mean_score_time                                                                             0.090346
std_score_time                                                                              0.065709
param_modelo                                                     KNeighborsClassifier(n_neighbors=6)
param_modelo__penalty                                                                            NaN
param_modelo__solver                                                                             NaN
param_percentiles__percentile                                                              

El mejor modelo obtenido es un KNeighborsClassifier con K=6 con Percentil=60 y Ngram=(1,2), el cual permite clasificar la mayoría de los registros dado que posee un alto K pero que toma las carácteristicas principales de los datos, ya que usa el percentil 60 y un enegrama de 1,2, indicando que solo toma las palabras más importantes de cada registro.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [41]:
comics_to_clas = df_comics_no_label.dropna(subset=["history_text"]).drop_duplicates(keep = 'first')
new_heroes = comics_to_clas[comics_to_clas['name'].isin(['Vergil', 'Gorilla Girl', 'Batcow'])]

In [43]:
X = new_heroes.drop('alignment', axis = 1)
y_pred = grid_model.predict(X)

In [45]:
X['alignment_pred'] = y_pred
X[['name', 'alignment_pred']].head()

,name,alignment_pred
16,Batcow,Good
40,Gorilla Girl,Good
78,Vergil,Good


Los tres sujetos en cuestión son heroes.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>